<a href="https://colab.research.google.com/github/Zhi-704/ERL/blob/master/QLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
NEED TO FIX CHOSEN ACTION AS CAN MAKE ILLEGAL MOVES
'''

In [ ]:
pip install jumanji

In [ ]:
import jumanji
import jax.numpy as jnp
import jax
import random
from collections import namedtuple, deque
import tensorflow as tf
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.layers import Dense
import numpy as np
import matplotlib.pyplot as plt
from time import sleep

In [ ]:
# Parameters for algorithm and agents generated
epsilon = 0.01
discount_factor = 0.99
learning_rate = 0.20

def get_action(state):
  array = np.array(state.action_mask)
  # Flatten the array
  flattened_array = array.flatten()
  # Get the indices of True values
  true_indices = np.where(flattened_array)[0]
  if len(true_indices) == 0:
   return False
  random_index = np.random.choice(true_indices)
  # Convert the random index to two-dimensional coordinates
  rotation, col_index = np.unravel_index(random_index, array.shape)
  return [rotation,col_index]

def get_valid_actions(state):
  array = np.array(state.action_mask)
  # Flatten the array
  flattened_array = array.flatten()
  # Get the indices of true values, which are the number of valid actions in the current state
  true_indices = np.where(flattened_array)[0]

  return true_indices

def choose_action(Qvalues,state, state_tuple):
  # # Makes sure the algorithm does not always pick the first index if all index values are the same
  # if np.all(Qvalues[state] == Qvalues[state][0]):
  #   return [(np.random.randint(0,3)), (np.random.randint(0,9))]
  valid_actions = get_valid_actions(state)
  if len(valid_actions) == 0:
    return False
  # Choose action with epsilon-greedy policy
  if np.random.uniform(0,1) < epsilon:
    action_index = np.random.choice(valid_actions, size = 1)
  else:
    # Access array associated with the state key and find the index of the maximum value
    index_with_highest_Q_value = np.argmax(Qvalues[state_tuple][:, 1])
    # Take only the valid action
    action_index = Qvalues[state_tuple][index_with_highest_Q_value, 0]

  # Convert the action index to two-dimensional coordinates
  rotation, col_index = np.unravel_index(action_index, array.shape)

  return [rotation, col_index], action_index


def Qalgorithm(Qvalues):

  # Initialises environment and rewards
  rewards = []
  # Reset your (jit-able) environment
  key = jax.random.PRNGKey(0)
  state, timestep = jax.jit(env.reset)(key)
  state_tuple = tuple(state)

  env.render(state)

  valid_actions = get_valid_actions(state)

  # Adds new state to Q values tables and model
  if state_tuple not in Qvalues:
    corresponding_array = [[x, 0] for x in valid_actions]
    Qvalues.update({state_tuple: corresponding_array})

  print("STATE UPDATED FIRST")
  print(Qvalues[state_tuple])

  while True:

    # Chooses max value action and takes the action
    chosen_action, action_index = choose_action(Qvalues, state, state_tuple)
    print("ACTION CHOSEN")
    # next_state, reward, terminal = env.step(chosen_action)
    next_state, timestep = jax.jit(env.step)(state, chosen_action)
    next_state_tuple = tuple(next_state)
    curr_reward = next_state.reward

    print("NEXT STATE UPDATED")

    new_valid_actions = get_valid_actions(next_state)
    # Adds new state to Q values and Model
    if next_state_tuple not in Qvalues:
      corresponding_array2 = [[x,0] for x in new_valid_actions]
      Qvalues.update({next_state_tuple: corresponding_array2})

    print("NEXT STATE UPDATED SECOND")
    print(Qvalues[next_state_tuple])

    # Adds reward to prior reward
    rewards.append(curr_reward)

    # Locates action index
    chosen_action_index = np.where(Qvalues[state_tuple][:,0] == action_index)[0]

    # Updates Q value table
    Qvalues[state_tuple][chosen_action_index][1] += learning_rate * (curr_reward + discount_factor * np.max(Qvalues[next_state_tuple][:, 1]) - Qvalues[state_tuple][chosen_action_index][1])
    print("Q VALUE UPDATED")
    print(Qvalues[state_tuple])
    # Ends episode once terminal state is reached
    if chosen_action is False:
        break

    print("TERMINAL PASSED")
    state = next_state
    state_tuple = next_state_tuple

    time.sleep(3)

    env.render(state)


  return sum(rewards)

In [ ]:
num_of_agents = 1

def generateagent(episodes):
  # Resets Q values and Model table
  Qvalues = {}
  # Generates an agent that follows QLearning for a number of episodes
  total_rewards = np.zeros(episodes)
  for i in range(len(total_rewards)):
      total_rewards[i] = Qalgorithm(Qvalues)
  return total_rewards